In [3]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os


In [4]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 6s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [6]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [8]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


In [9]:
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [10]:
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [12]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [13]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False, 
        zca_epsilon=1e-06,  
        rotation_range=0,  
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.,  
        zoom_range=0.,  
        channel_shift_range=0.,  
        fill_mode='nearest',
        cval=0.,  
        horizontal_flip=True,  
        vertical_flip=False,  
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

Using real-time data augmentation.


In [14]:
datagen.fit(x_train)

In [15]:
model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
1563/1563 [==============================] - 48s 31ms/step - loss: 1.8703 - accuracy: 0.3138 - val_loss: 1.5737 - val_accuracy: 0.4274
Epoch 2/100
1563/1563 [==============================] - 48s 31ms/step - loss: 1.5833 - accuracy: 0.4205 - val_loss: 1.4035 - val_accuracy: 0.4891
Epoch 3/100
1563/1563 [==============================] - 47s 30ms/step - loss: 1.4655 - accuracy: 0.4674 - val_loss: 1.2857 - val_accuracy: 0.5421
Epoch 4/100
1563/1563 [==============================] - 47s 30ms/step - loss: 1.3788 - accuracy: 0.5046 - val_loss: 1.2832 - val_accuracy: 0.5350
Epoch 5/100
1563/1563 [==============================] - 47s 30ms/step - loss: 1.3152 - accuracy: 0.5294 - val_loss: 1.1649 - val_accuracy: 0.5896
Epoch 6/100
1563/1563 [==============================] - 48s 31ms/step - loss: 1.2568 - accuracy: 0.5510 - val_loss: 1.0816 - val_accuracy: 0.6152
Epoch 7/100
1563/1563 [===================

In [17]:
model.evaluate(x_train,y_train)

1563/1563 [==============================] - 8s 5ms/step - loss: 0.6217 - accuracy: 0.7939


[0.6217466592788696, 0.7938799858093262]

In [18]:
score = model.evaluate(x_test,y_test)

313/313 [==============================] - 2s 5ms/step - loss: 0.6891 - accuracy: 0.7708


In [19]:
print('Test Loss:',score[0])
print('Test Accuracy:',score[1])

Test Loss: 0.6891160011291504
Test Accuracy: 0.770799994468689
